# Tutorial for using quante_carlo for hyperparameter tuning
quante_carlo is a hyperparameter tuning module that has integrated multiprocessing support
It is able to use cpu or gpu.
The algorithm for hp tuning is Bayesian Optimization and with a <a href="https://scikit-learn.org/stable/modules/gaussian_process.html">gaussian process regressor</a>.

### Parameters
quante.carlo(f, limits, kernel, n_batches, n_processors, n_iterations, keep_thread_id=False)
- <b>f </b> evaluation function
- <b>limits</b>list of parameter ranges
- <b>kernel</b>
- <b>n_batches</b>  number of batches to use when
- <b>n_processors</b> number of processors; should align with actual hardware
- <b>n_iterations</b> number of iterations to run

## Load the module and then tune Sci-kit's Multi-Layer Perceptron and Logistic Regression

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import pickle
import multiprocessing as mp
import pandas as pd
from sklearn import datasets
from quante_carlo import quante
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF, Matern
import time
import warnings
import numpy as np

### Iris dataset
Convert data to files
When multiprocessing, it's better for each process to read the data by itself. There's less IPC, less blocking.
This code writes the dataset to a file in this directory

In [ ]:
iris = datasets.load_iris()
pd.DataFrame(iris.data).to_csv('X.csv', index=False)
with open('y.csv', 'w') as f:
    f.write(','.join([str(x) for x in iris.target]))

## User Define Valutation Wrapper for MLPClassifier
- notice that the input is the valution function is a list; the parameters are then placed into the MLPClassifier constructor
- multiprocessing works best when in main

In [ ]:
if __name__ == '__main__':
    
    def instance(arch):
    
        X = pd.read_csv('X.csv')
        with open('y.csv') as f:
            y = f.read().split(',')
            
        clf = MLPClassifier(solver='sgd', max_iter=500,
                        hidden_layer_sizes=arch[:3], alpha=arch[3], random_state=1)
        with warnings.catch_warnings():
            warnings.simplefilter('ignore') # convergence warning
            cvs = cross_val_score(clf, X, y)
        return cvs
            
    hp_tune = quante.carlo(instance, [[2, 16], [2, 4], [2, 16], [.01, .99]], kernel=DotProduct()+ WhiteKernel(), 
                           n_batches=100, n_processors=4, n_iterations=5)
    
    p = mp.Pool()
    start = time.time()
    
    session = hp_tune(p)
    print("{} seconds".format(round(time.time() - start,2)))
    p.close()




In [ ]:
summary = session.summary()
summary.head()

In [ ]:
n = summary[['score', 'iteration']].groupby('iteration').mean().plot()

## Box and Whiskers plot
round alpha so that it prints a little better on the x-axis

In [ ]:
rounded_alpha =  [x[:3]+(round(x[3],3),) for x in summary['layers']]
fig = plt.figure(figsize =(12, 6))
ax = fig.add_subplot(111)
ax.boxplot(session.score_history)
ax.set_xticklabels(rounded_alpha)
plt.xticks(rotation=60)
bx = plt.show


## Example using Logistic Regression

In [ ]:
if __name__ == '__main__':
    
    def instance(hp_parameter):
    
        X = pd.read_csv('X.csv')
        with open('y.csv') as f:
            y = f.read().split(',')
            
        clf = LogisticRegression(solver='saga', penalty = 'elasticnet', l1_ratio=hp_parameter[0], max_iter=200 )
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            cvs = cross_val_score(clf, X, y)
        
        return cvs
        
    
    hp_tune = quante.carlo(instance, [[.05, .95]], kernel=DotProduct()+ WhiteKernel(), n_batches=7, n_processors=4, n_iterations=5)
    p = mp.Pool()
    start = time.time()
    
    session = hp_tune(p)
    print("{} seconds".format(round(time.time() - start,2)))
    p.close()
